In [ ]:
### old structure ###
class Initiatives(Base):
    __tablename__ = 'initiatives' 
    
    initiative_id = Column(Integer, primary_key=True)
    initiative_name = Column(String(250), nullable=False)
    act_type = Column(String(10), nullable=False)
    reference_document = Column(String(50))
    dossier_summary = Column(Text)
    published_date = Column(DateTime)
    initiative_status = Column(String(10))
    current_stage = Column(String(20))
    dg = Column(String(5))
    receiving_feedback = Column(String(10))
    better_regulation_requirements = Column(String(100))
    topic = Column(String(250))
    db_update = Column(DateTime)

### new structure ####
class Initiatives(Base):
    __tablename__ = 'initiatives' 
    
    initiative_id = Column(Integer, primary_key=True)
    initiative_name = Column(String(250), nullable=False)
    act_type = Column(String(10), nullable=False)
    reference_document = Column(String(50))
    dossier_summary = Column(Text)
    published_date = Column(DateTime)
    initiative_status = Column(String(10))
    current_stage = Column(String(20))
    dg = Column(String(5))
    receiving_feedback = Column(String(10))
    better_regulation_requirements = Column(String(100))
    topic = Column(String(250))

####
#### old
class Stages(Base):
    __tablename__ = 'stages'  
    stage_id = Column(Integer, primary_key=True)
    type = Column(String(20), nullable=False)
    total_feedback = Column(Integer, nullable=False)
    published_date = Column(DateTime)
    end_date = Column(DateTime)
    receiving_feedback = Column(String(10))
    initiative_id = Column(Integer, nullable=False)
    db_update = Column(DateTime)

### new
class Stages(Base):
    __tablename__ = 'stages'
    id = Column(String(100), primary_key=True)
    stage_id = Column(Integer, nullable=False)
    type = Column(String(20), nullable=False)
    total_feedback = Column(Integer, nullable=False)
    published_date = Column(DateTime)
    end_date = Column(DateTime)
    receiving_feedback = Column(String(10))
    initiative_id = Column(Integer, nullable=False)
    feedback_updated = Column(DateTime)
    attachment_updated = Column(DateTime)

####
#### old  
class Feedbacks(Base):
    __tablename__ = 'feedbacks'  
    feedback_id = Column(Integer, primary_key=True)
    language = Column(String(2), nullable=False)
    country = Column(String(3), nullable=False)
    organization = Column(String(300), nullable=False)
    tr_id = Column(String(250))
    feedback = Column(Text, nullable=False)  # Text is used for longer string-based data
    date_feedback = Column(DateTime)
    is_anonymous = Column(Boolean)
    user_type = Column(String(30))
    company_size = Column(String(100))
    scope = Column(String(100))
    governance_level = Column(String(100))
    hash_name = Column(BigInteger)
    stage_id = Column(Integer, nullable=False)
    db_update = Column(DateTime)

#### new
class Feedbacks(Base):
    __tablename__ = 'feedbacks'  
    id = Column(String(100), primary_key=True)
    feedback_id = Column(Integer, nullable=False)
    language = Column(String(2), nullable=False)
    country = Column(String(3))
    organization = Column(String(300))
    tr_id = Column(String(250))
    feedback = Column(Text, nullable=False)  # Text is used for longer string-based data
    date_feedback = Column(DateTime)
    is_anonymous = Column(Boolean)
    user_type = Column(String(30))
    company_size = Column(String(100))
    scope = Column(String(100))
    governance_level = Column(String(100))
    hashed_name = Column(BigInteger)
    stage_id = Column(Integer, nullable=False)
    attachments = Column(Integer, nullable=False)

####
#### old
class TextFileContents(Base):
    __tablename__ = 'attachments'
    unique_id = Column(String(50), primary_key=True)
    feedback_id = Column(Integer, nullable=False)
    attachment_number = Column(Integer, nullable=False)
    stage_id = Column(Integer, nullable=False)
    content = Column(LONGTEXT, nullable=False)
    db_update = Column(DateTime)

#### new
class Attachments(Base):
    __tablename__ = 'attachments'
    document_id = Column(String(100), primary_key=True)
    attachment_id = Column(Integer, nullable=False)
    feedback_id = Column(Integer, nullable=False)
    stage_id = Column(Integer, nullable=False)
    content = Column(LONGTEXT)
    file = Column(String(250))

####
####
#### new

class SeedList(Base):
    __tablename__ = 'seed_list'
    initiative_id = Column(Integer, primary_key=True)
    initiative_name = Column(String(250), nullable=False)
    seed_list_updated = Column(DateTime)
    initiative_updated = Column(DateTime)


# Migrate Initiatives

In [8]:
## check if all old stages are in new DB
import pandas as pd
import numpy as np

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import update

from time import gmtime, strftime
from src.database.database_structure import Base

from database_structure import Feedbacks
from database_connection import database_connection as con_new
from database_connection_old import database_connection as con_old


### feedback migration
conn_old = create_engine(con_old(), echo=False, echo_pool=False)
conn_new = create_engine(con_new(), echo=False, echo_pool=False)

initiative_ids_old = pd.read_sql("SELECT initiative_id FROM initiatives", conn_old)
initiative_ids_new = pd.read_sql("SELECT initiative_id FROM seed_list", conn_new)

In [10]:
merged = pd.merge(initiative_ids_old.reset_index(), initiative_ids_new.reset_index(), on="initiative_id", how='outer', indicator=True)
print(merged.value_counts("_merge"))

_merge
right_only    501
both          205
left_only       9
Name: count, dtype: int64


In [11]:
display(merged[merged._merge == "left_only"])

,index_x,initiative_id,index_y,_merge
385,36.0,12658,NaN,left_only
404,49.0,12733,NaN,left_only
436,66.0,12827,NaN,left_only
518,111.0,13128,NaN,left_only
534,121.0,13169,NaN,left_only
543,126.0,13188,NaN,left_only
575,155.0,13321,NaN,left_only
611,188.0,13488,NaN,left_only
632,204.0,13611,NaN,left_only


# Migrate Stages

In [12]:
## check if all old stages are in new DB
import pandas as pd
import numpy as np

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import update

from time import gmtime, strftime
from src.database.database_structure import Base

from database_structure import Feedbacks
from database_connection import database_connection as con_new
from database_connection_old import database_connection as con_old


### feedback migration
conn_old = create_engine(con_old(), echo=False, echo_pool=False)
conn_new = create_engine(con_new(), echo=False, echo_pool=False)

sql = """
SELECT stage_id
FROM stages
"""
stage_ids_old = pd.read_sql(sql, conn_old)
stage_ids_new = pd.read_sql(sql, conn_new)

In [13]:
merged = pd.merge(stage_ids_old.reset_index(), stage_ids_new.reset_index(), on="stage_id", how='outer', indicator=True)
print(merged.value_counts("_merge"))

_merge
right_only    935
both          738
left_only       0
Name: count, dtype: int64


# Migrate Feedbacks

In [ ]:
def upsert_feedbacks_to_database(row, Feedbacks):
    global s
    '''
    INSERT INTO my_table (id, data) VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE data = VALUES(data), status = %s
    '''
    feedback_dict = row.to_dict()
    
    insert_stmt = insert(Feedbacks).values(
                                            id          =           str(feedback_dict["stage_id"]) + "_" + str(feedback_dict["feedback_id"]),
                                            feedback_id =           int(feedback_dict["feedback_id"]),
                                            language =              feedback_dict.get("language", None),
                                            country =               feedback_dict.get("country", None),
                                            organization =          feedback_dict.get("organization", None),
                                            tr_id =                 feedback_dict.get("tr_id", None),
                                            feedback =              feedback_dict.get("feedback", None),
                                            date_feedback =         feedback_dict["date_feedback"],
                                            is_anonymous =          feedback_dict["is_anonymous"],
                                            user_type =             feedback_dict.get("user_type", None),
                                            company_size =          feedback_dict.get("company_size", None),
                                            scope =                 feedback_dict.get("scope", None),
                                            governance_level =      feedback_dict.get("governance_level", None),
                                            hashed_name =           feedback_dict["hash_name"],
                                            stage_id =              int(feedback_dict["stage_id"]),
                                            attachments =           feedback_dict.get("attachments", None) # not in old DB
                                            )
    
    on_duplicate_key_stmt = insert_stmt.on_duplicate_key_update(attachments = insert_stmt.inserted.attachments)
    s.execute(on_duplicate_key_stmt)
    return None


In [ ]:
def add_nr_of_attachments(feedback_df, con_old):
    # count attachments
    sql = """
        SELECT *
        FROM attachments
        """

    attachments_df = pd.read_sql(sql, conn_old)
    
    vc = attachments_df["feedback_id"].value_counts()
    vc_df = pd.DataFrame({"feedback_id" : vc.index, "attachments": vc.values})
    vc_df = vc_df.astype({'attachments': 'int'})
    
    # add to feedback df
    feedback_df = pd.merge(feedback_df, vc_df, on='feedback_id', how='outer')
    
    return feedback_df

In [10]:
import pandas as pd
import numpy as np

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import update

from time import gmtime, strftime
from src.database.database_structure import Base

from database_structure import Feedbacks
from database_connection import database_connection as con
from database_connection_old import database_connection as con_old


### feedback migration
conn_old = create_engine(con_old(), echo=False, echo_pool=False)
conn_new = create_engine(con(), echo=False, echo_pool=False)

sql = """
SELECT *
FROM feedbacks
"""

feedback_df = pd.read_sql(sql, conn_old)
# remember ids for later
migrated_ids = feedback_df["feedback_id"]

# check how many attachments per feedback
feedback_df = add_nr_of_attachments(feedback_df, conn_old)

# convert nan to None
feedback_df.replace({np.nan: None}, inplace=True)
feedback_df.replace({"nan": None}, inplace=True)

# convert None to 0 for attachments column
feedback_df["attachments"].replace({None: 0}, inplace=True)



# open Session with new DB
Session = sessionmaker(bind=conn_new)
session = Session()
global s
with session as s:
    
    for i, row in feedback_df.tail(-110000).iterrows():
        upsert_feedbacks_to_database(row, Feedbacks)
        
        
        if i % 1000 == 0:
            print(str(i)+"/"+str(len(feedback_df.index)))
            s.commit()
        


#upsert_feedbacks_to_database(feedback_df, sess, Feedbacks)

/var/folders/vv/zkl2vlbx3js01k9f_541yjq80000gn/T/ipykernel_3786/3418227363.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feedback_df["attachments"].replace({None: 0}, inplace=True)
/var/folders/vv/zkl2vlbx3js01k9f_541yjq80000gn/T/ipykernel_3786/3418227363.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcas

110000/146385
111000/146385
112000/146385
113000/146385
114000/146385
115000/146385
116000/146385
117000/146385
118000/146385
119000/146385
120000/146385
121000/146385
122000/146385
123000/146385
124000/146385
125000/146385
126000/146385
127000/146385
128000/146385
129000/146385
130000/146385
131000/146385
132000/146385
133000/146385
134000/146385
135000/146385
136000/146385
137000/146385
138000/146385
139000/146385
140000/146385
141000/146385
142000/146385
143000/146385
144000/146385
145000/146385
146000/146385


In [ ]:
# attachments




In [3]:
import pandas as pd
import numpy as np
import datetime

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import update

from time import gmtime, strftime
from src.database.database_structure import Base

from database_structure import Feedbacks, Stages
from database_connection import database_connection as con
from database_connection_old import database_connection as con_old

In [10]:
# update datetime in stages table
def update_seedlist_feedback_scrape(stage_id:int):
    global s
    d = datetime.datetime(1111, 1, 1, 1, 1, 1) 
    current_time = d.strftime("%Y-%m-%d %H:%M:%S")
    
    update_stmt = update(Stages).where(Stages.stage_id == stage_id).values(feedback_updated=current_time)
    s.execute(update_stmt)

### feedback migration
conn_old = create_engine(con_old(), echo=False, echo_pool=False)
conn_new = create_engine(con(), echo=False, echo_pool=False)

# update which ids? 
sql = """
SELECT *
FROM feedbacks
"""
feedback_df = pd.read_sql(sql, conn_old)
migrated_ids = feedback_df["stage_id"].unique()
migrated_ids = migrated_ids.tolist()

# open Session with new DB
Session = sessionmaker(bind=conn_new)
session = Session()
global s
with session as s:
    c = 1
    for id in migrated_ids:
        update_seedlist_feedback_scrape(int(id))
        c += 1
        if c%20 == 0:
            print("{}/{}".format(c, len(migrated_ids)))
    s.commit()

20/361
40/361
60/361
80/361
100/361
120/361
140/361
160/361
180/361
200/361
220/361
240/361
260/361
280/361
300/361
320/361
340/361
360/361
